This code takes in articles and outputs the corresponding layperson summarizations using ChatGPT 3.5.

This was run in Google Colab on a V100.

In [ ]:
# install pip dependencies

%pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


'\n%pip install ttok\n!ttok articles[7]\n'

In [ ]:
# access OpenAI client

OPENAI_API_KEY='YOUR_API_KEY'
PROJECT_ID='YOUR_PROJECT_ID'

from openai import OpenAI
client = OpenAI(project=PROJECT_ID,
                api_key=OPENAI_API_KEY
                )

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount("/content/drive")

df = pd.read_json("/content/drive/MyDrive/CPSC_477/CPSC477_CHAD/biolaysumm2024_data/PLOS_val.jsonl", lines=True) # your directory


Mounted at /content/drive


In [ ]:
def gen_laysumm(article):
  """ Takes article and returns lay summary."""

  # instruct the model how to summarize
  INPUT_TEXT = "Summarise the following article for a non-expert audience. Please: 1. Replace arcane words with common synonyms. 2. Split long, complex sentences into shorter, simpler sentences. 3. Omit experimental results that are too detailed for lay readers, like confidence intervals and other statistical values. 4. Add explanations for complex terms and abbreviations in the article. 5. The summarization should be one paragraph. \n\
  Article: "+article

  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a reading assistant, skilled in explaining complex scientific articles in simple terms."},
      {"role": "user", "content": INPUT_TEXT}
    ],
  )

  return completion.choices[0].message.content

In [ ]:
from tqdm import tqdm

articles = df[df.columns[1]]

for article in tqdm(articles):
  if len(article) > 64000: # crude way to cap input
    article = article[:64000]
  summary = gen_laysumm(article)
  # open existing text file
  with open('/content/drive/MyDrive/CPSC_477/CPSC477_CHAD/outputs_for_eval/chatgpt_val_outputs.txt', 'a') as f: # your directory
    f.write(summary)
    f.write('\n')

 19%|█▉        | 265/1376 [39:18<2:44:48,  8.90s/it]


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 16669 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}